<a href="https://colab.research.google.com/github/peisuke/generative_ai_notebooks/blob/main/08_PLaMo_13B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate safetensors sentencepiece bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.8 MB/s eta 0:00:00


In [2]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="pfnet/plamo-13b-instruct")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

LICENSE.txt:   0%|          | 0.00/23.0k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/783 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.35G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--pfnet--plamo-13b-instruct/snapshots/40028341baf7e9da1f6deceab2815490575fe046'

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "pfnet/plamo-13b-instruct",
    trust_remote_code=True,
)
model = AutoModelForCausalLM.from_pretrained(
    "pfnet/plamo-13b-instruct",
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto",
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# プロンプトの準備
prompt = """以下の質問に対し、回答を書きなさい。

### 質問:
機械学習って何？

### 回答: """

# 推論の実行
token_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)
output_ids = model.generate(
    token_ids.to(device=model.device),
    eos_token_id=2,
    pad_token_id=3,
    max_new_tokens=128,
    temperature=1,
    top_p=0.95,
    top_k=50,
    do_sample=True,
)
output = tokenizer.decode(
    output_ids[0][token_ids.size(1) :],
    skip_special_tokens=True
)

In [6]:
print(output)


機械学習(machine learning)とは、統計的モデルをアルゴリズムで実行し、予測し、機械学習アルゴリズムは、アルゴリズムで実行し、予測し、機械学習アルゴリズムは、アルゴリズムで実行し、予測し、機械学習アルゴリズムは、アルゴリズムで実行し、予測し、機械学習アルゴリズムは、アルゴリズムで実行し、予測し、機械学習アルゴリズムは、アルゴリズムで実行し、予測し、機械学習アルゴリズムは、アルゴリズムで実行し、予測し、機械学習アルゴリズムは、アルゴリズムで実行し、予測し、機械学習アルゴリズムは、アルゴリズムで実行し、予測し、機械学習
